<a href="https://colab.research.google.com/github/ShadmanRohan/n-gram-language-model/blob/master/ngam-language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Embedding Layer in PyTorch

In [2]:
word_to_ix = {"hello": 0, "world": 1} # put two words in a dictionary
embeds = nn.Embedding(3, 5)  # 2 words in vocab, 5 dimensional embeddings
print(embeds)
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long) # torch.long is an Integer, returns the index
print(lookup_tensor)
hello_embed = embeds(torch.tensor([0])) # callable returns the vector in the index
#print(torch.tensor([4]))
print(hello_embed) # randomly generated vector

Embedding(3, 5)
tensor([1])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [3]:
embedding = nn.Embedding(1000,128) # randomly initialised matrix, 10000 words * 128 dim each
print(embedding(torch.LongTensor([3,6]))) # vocabulary indexed at 3 and 6
print(embedding(torch.LongTensor([6]))) # vocabulary index at 6 only


tensor([[-4.5518e-01,  3.1860e-01, -3.5494e-01,  6.8589e-01, -3.7614e-01,
         -2.4107e+00, -1.2778e+00, -6.2887e-02, -9.4713e-02, -2.3144e+00,
          5.5653e-01,  5.0569e-01, -2.0760e-01,  6.9363e-01,  4.1949e-01,
          2.2524e+00,  9.3852e-01,  1.4253e+00,  1.5083e+00,  1.0539e-01,
         -1.6050e+00, -1.0645e-01,  2.4657e-01,  6.1251e-01,  7.3980e-01,
         -1.7860e-01,  7.8490e-02, -4.3982e-01, -3.6079e-01, -1.2617e+00,
          1.9147e+00, -1.8613e+00, -9.6749e-03,  2.6039e-01,  2.8203e-01,
          2.5830e-01, -4.2655e-01,  9.8075e-01,  1.8589e+00, -1.0920e+00,
          7.6300e-01,  2.2762e-01, -1.4570e+00,  1.7044e+00, -3.2686e+00,
          4.7499e-01, -2.1142e+00, -1.5002e+00,  1.0693e+00,  1.4394e+00,
          5.0646e-01,  8.3598e-01,  1.1753e+00, -3.4212e-01, -3.8716e-01,
          5.4765e-01, -1.5892e-01, -7.3605e-01, -2.3352e-01, -5.4039e-01,
          1.5708e-01, -5.9762e-01, -8.8391e-01,  6.0767e-01, -3.8844e-01,
         -3.1579e-02, -5.6059e-01, -6.

### Train a Tri-gram Language Model

#### 1. Prepare data

In [4]:
CONTEXT_SIZE = 2 # parameters
EMBEDDING_DIM = 10 # word embedding dimension
# Some random bangla news paper article
test_sentence = """প্রথমেই শুরু হয়েছে স্বাস্থ্য ও চিকিৎসার সংকট। এতে সরকারকে জরুরি স্বাস্থ্য বরাদ্দ বাড়াতে হবে। অন্যথায় স্বাস্থ্য ও চিকিৎসার 
সংকট প্রলম্বিত হয়ে অনুৎপাদনশীলতার জন্ম হবে, যার আর্থিক দায় ব্যাপক। দ্বিতীয় সংকট হতে পারে খাদ্য ও মানবিক সংকট। যেহেতু সংক্রমণ বিস্তার 
রোধে প্রায় পূর্ণাঙ্গ ‘লকডাউন’ শুরু হয়েছে, তাই নিম্ন আয়ের মানুষ অর্থ ও সঞ্চয় সংকটে পড়বে। শুরুতেই সঞ্চয়হীন ভাসমান মানুষ, দিনমজুর, 
বৃদ্ধ-অনাথ-এতিম, রিকশা, ছোট কারখানা, নির্মাণশ্রমিক যাঁরা ‘দিন আনে দিন খান’, তাঁরা লকডাউনের দ্বিতীয় সপ্তাহ থেকেই আয় হীনতার কারণে 
খাদ্যের সংকটে পড়বেন। শহরের ভাসমান প্রান্তিক অর্থনৈতিক শ্রেণি সামাজিক উৎস থেকে ধার-ঋণ নিতে অক্ষম বলে তাদের জন্য খাদ্যসংকট অবধারিত। 
গ্রামে ‘সমাজের’ উপস্থিতি এবং কৃষি ও ক্ষুদ্রশিল্পভিত্তিক ‘উৎপাদনব্যবস্থা’ রয়েছে বিধায় সেখানে খাদ্যসংকট কিছুটা দেরিতে আসবে। গ্রামে ভাসমানদের 
কর্মহীনতার তৃতীয় কিংবা চতুর্থ সপ্তাহ থেকে খাদ্যসংকট শুরু হতে পারে, তার আগে পর্যন্ত তাঁরা চেয়েচিন্তে চলতে পারবেন হয়তো। পরেই আসবে 
কর্মহীন নিম্নবিত্ত, যাদের কিছুটা সঞ্চয় ছিল—এমন শ্রেণি। তার পরে আসবে বেতন বন্ধ হয়ে সঞ্চয় ফুরিয়ে যাওয়া নিম্ন–মধ্যবিত্ত কিংবা মধ্যবিত্তও। 
এই সব কটি প্রান্তিক ধারার জন্য জরুরি খাদ্য সরবরাহ করার একটা দায় আছে সরকারের। ইতিমধ্যেই পশ্চিমবঙ্গ সরকার সাত কোটি মানুষের 
ছয় মাসের জরুরি খাদ্য সরবরাহের ঘোষণা দিয়েছে। অন্যদিকে কেরালার সরকার কুড়ি হাজার কোটি রুপির করোনা প্যাকেজ ঘোষণা করেছে, 
এসেছে বিদ্যুৎ বিলে ছাড়ের ঘোষণা। বাংলাদেশেও মাথাপিছু ন্যূনতম ‘ক্যালরি ধারণ’ ভিত্তিতে ভাসমান প্রান্তিক শ্রেণি, স্থায়ী বেকার, তাৎক্ষণিকভাবে 
কাজহীন, বেতন বন্ধ হয়ে পড়া, সঞ্চয় ফুরিয়ে যাওয়া শ্রেণির জন্য খাদ্য সরবরাহের বাধ্যবাধকতা তৈরি হয়েছে। আর্থিক সংখ্যায় রূপান্তর করলে দেখা 
যায়, সরকারের জন্য তৈরি হয়েছে বড় এক আর্থিক বোঝা!

""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:5])

vocab = set(test_sentence) # store the distinct words
word_to_ix = {word: i for i, word in enumerate(vocab)}
#print(torch.LongTensor([1,2,3]))

[(['প্রথমেই', 'শুরু'], 'হয়েছে'), (['শুরু', 'হয়েছে'], 'স্বাস্থ্য'), (['হয়েছে', 'স্বাস্থ্য'], 'ও'), (['স্বাস্থ্য', 'ও'], 'চিকিৎসার'), (['ও', 'চিকিৎসার'], 'সংকট।')]


#### 2. Create and Train Model [Embedding Froozen]

In [8]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1)) # view flattens the input
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
print("Before Training : {}".format(model.embeddings(torch.LongTensor([1,2])))) # first few random word embeddings
optimizer = optim.SGD(model.parameters(), lr=0.001)
model.embeddings.weight.requires_grad = False
for epoch in range(200):
    total_loss = 0
    for context, target in trigrams:

        # Prepare the inputs to be passed to the model
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long) #turn the words into integer indices and wrap them in tensors

        # Accumulated Gradient must be zeroed out before computing gradient again
        model.zero_grad()

        # Forward pass(probably using __Callable__), returns log probabilities over next words
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))  #target word wrapped in a tensor

        # Let the gradient flow backward
        loss.backward()
        
        # Update the gradient
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    
#print(losses)  # The loss decreased every iteration over the training data!

print("After Training : {}".format(model.embeddings(torch.LongTensor([1,2])))) # first few random word embeddings again

Before Training : tensor([[-1.9708, -2.2711, -2.3553, -0.0766,  0.2062, -0.1147,  0.2659, -0.5558,
          0.2354, -0.7534],
        [ 0.2136, -0.8176, -0.2218,  1.5490, -2.0157,  1.1105, -0.2885,  1.0748,
         -1.5732,  0.6442]], grad_fn=<EmbeddingBackward>)
After Training : tensor([[-1.9708, -2.2711, -2.3553, -0.0766,  0.2062, -0.1147,  0.2659, -0.5558,
          0.2354, -0.7534],
        [ 0.2136, -0.8176, -0.2218,  1.5490, -2.0157,  1.1105, -0.2885,  1.0748,
         -1.5732,  0.6442]])


#### 2. Create and Train Model [Embedding Trainable]

In [10]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1)) # view flattens the input
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
print("Before Training : {}".format(model.embeddings(torch.LongTensor([1,2])))) # first few random word embeddings
optimizer = optim.SGD(model.parameters(), lr=0.001)
# model.embeddings.weight.requires_grad = False
for epoch in range(200):
    total_loss = 0
    for context, target in trigrams:

        # Prepare the inputs to be passed to the model
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long) #turn the words into integer indices and wrap them in tensors

        # Accumulated Gradient must be zeroed out before computing gradient again
        model.zero_grad()

        # Forward pass(probably using __Callable__), returns log probabilities over next words
        log_probs = model(context_idxs)

        # Compute loss function
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))  #target word wrapped in a tensor

        # Let the gradient flow backward
        loss.backward()
        
        # Update the gradient
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    
#print(losses)  # The loss decreased every iteration over the training data!

print("After Training : {}".format(model.embeddings(torch.LongTensor([1,2])))) # first few random word embeddings again

Before Training : tensor([[ 0.9915,  0.6616, -0.3415, -0.8382, -1.0565,  0.9416,  0.6389, -0.3896,
         -0.1721, -2.0031],
        [ 3.1797, -1.1324, -0.9324,  1.3209,  0.8647, -0.5295, -0.4210, -1.0535,
          0.2932, -0.2520]], grad_fn=<EmbeddingBackward>)
After Training : tensor([[ 1.0526,  0.6743, -0.3418, -0.8707, -1.0880,  0.9743,  0.6368, -0.3906,
         -0.1742, -2.0610],
        [ 3.2230, -1.1339, -0.9538,  1.3542,  0.9175, -0.5268, -0.4367, -1.0652,
          0.3058, -0.2182]], grad_fn=<EmbeddingBackward>)
